In [9]:
import pandas as pd
import sys
from dwave.system import LeapHybridSampler
from math import log2, floor
import dimod
import numpy as np


In [10]:


def build_knapsack_bqm(costs, weights, weight_capacity):
    """Construct BQM for the knapsack problem
    Args:
        costs (array-like):
            Array of costs associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
    Returns:
        Binary quadratic model instance
    """

    # Initialize BQM - use large-capacity BQM so that the problem can be
    # scaled by the user.
    bqm = dimod.AdjVectorBQM(dimod.Vartype.BINARY)

    # Lagrangian multiplier
    # First guess as suggested in Lucas's paper
    lagrange = max(costs)

    # Number of objects
    x_size = len(costs)

    # Lucas's algorithm introduces additional slack variables to
    # handle the inequality. M+1 binary slack variables are needed to
    # represent the sum using a set of powers of 2.
    M = floor(log2(weight_capacity))
    num_slack_variables = M + 1

    # Slack variable list for Lucas's algorithm. The last variable has
    # a special value because it terminates the sequence.
    y = [2**n for n in range(M)]
    y.append(weight_capacity + 1 - 2**M)

    # Hamiltonian xi-xi terms
    for k in range(x_size):
        bqm.set_linear('x' + str(k), lagrange * (weights[k]**2) - costs[k])

    # Hamiltonian xi-xj terms
    for i in range(x_size):
        for j in range(i + 1, x_size):
            key = ('x' + str(i), 'x' + str(j))
            bqm.quadratic[key] = 2 * lagrange * weights[i] * weights[j]

    # Hamiltonian y-y terms
    for k in range(num_slack_variables):
        bqm.set_linear('y' + str(k), lagrange * (y[k]**2))

    # Hamiltonian yi-yj terms
    for i in range(num_slack_variables):
        for j in range(i + 1, num_slack_variables):
            key = ('y' + str(i), 'y' + str(j))
            bqm.quadratic[key] = 2 * lagrange * y[i] * y[j]

    # Hamiltonian x-y terms
    for i in range(x_size):
        for j in range(num_slack_variables):
            key = ('x' + str(i), 'y' + str(j))
            bqm.quadratic[key] = -2 * lagrange * weights[i] * y[j]

    return bqm

def solve_knapsack(costs, weights, weight_capacity, sampler=None):
    """Construct BQM and solve the knapsack problem
    Args:
        costs (array-like):
            Array of costs associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
        sampler (BQM sampler instance or None):
            A BQM sampler instance or None, in which case
            LeapHybridSampler is used by default
    Returns:
        Tuple:
            List of indices of selected items
            Solution energy
    """
    bqm = build_knapsack_bqm(costs, weights, weight_capacity)

    if sampler is None:
        sampler = LeapHybridSampler()

    sampleset = sampler.sample(bqm, label='Example - Knapsack')
    sample = sampleset.first.sample
    energy = sampleset.first.energy

    # Build solution from returned binary variables:
    selected_item_indices = []
    for varname, value in sample.items():
        # For each "x" variable, check whether its value is set, which
        # indicates that the corresponding item is included in the
        # knapsack
        if value and varname.startswith('x'):
            # The index into the weight array is retrieved from the
            # variable name
            selected_item_indices.append(int(varname[1:]))

    return sorted(selected_item_indices), energy


In [15]:
data_file_name = "dataset_knapsack.csv"
weight_capacity = 200

# parse input data
df = pd.read_csv(data_file_name, names=['cost', 'weight'])
print(df)
selected_item_indices, energy = solve_knapsack(df['cost'], df['weight'], weight_capacity)
selected_weights = list(df.loc[selected_item_indices,'weight'])
selected_costs = list(df.loc[selected_item_indices,'cost'])

print("Found solution at energy {}".format(energy))
print("Selected item numbers (0-indexed):", selected_item_indices)
print("Selected item weights: {}, total = {}".format(selected_weights, sum(selected_weights)))
print("Selected item costs: {}, total = {}".format(selected_costs, sum(selected_costs)))

    cost  weight
0     77      95
1     44      70
2     15      85
3     67      31
4     75     100
..   ...     ...
63    88      23
64     8      73
65     1      41
66    71      13
67    36      44

[68 rows x 2 columns]
Found solution at energy -884.0
Selected item numbers (0-indexed): [5, 9, 12, 15, 24, 29, 36, 46, 54, 55, 62, 63, 66]
Selected item weights: [1, 11, 4, 22, 24, 15, 20, 17, 19, 14, 10, 23, 13], total = 193
Selected item costs: [91, 68, 84, 43, 49, 18, 78, 23, 96, 83, 92, 88, 71], total = 884


### Using asset class dataset provided by Justin

In [16]:
data_file_name = "dataset_assetclass.csv"
weight_capacity = 10000

# parse input data
df = pd.read_csv(data_file_name)

df_renamed = df.rename(columns={"mean": "weight"})

selected_item_indices, energy = solve_knapsack(df_renamed['cost'], df_renamed['weight'], weight_capacity)
selected_weights = list(df_renamed.loc[selected_item_indices,'weight'])
selected_costs = list(df_renamed.loc[selected_item_indices,'cost'])

print("Found solution at energy {}".format(energy))
print("Selected item numbers (0-indexed):", selected_item_indices)
print("Selected item weights: {}, total = {}".format(selected_weights, sum(selected_weights)))
print("Selected item costs: {}, total = {}".format(selected_costs, sum(selected_costs)))


a = np.array(selected_costs)
b = np.array(selected_weights)
print(np.sum(a * b))


Found solution at energy -6882.658996582031
Selected item numbers (0-indexed): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Selected item weights: [0.06071298, 0.01215188, 0.02904863, 0.03567517, -0.00694793, 0.01725811, -0.00984779, 0.00965796, 0.02258329, -0.002925746, 0.077504005, -0.004550706, -0.00306835, 0.065000495, 0.000559336, 0.027820454, 0.068750039, 0.000442789, 0.035818423], total = 0.435643039
Selected item costs: [598.186, 188.194, 462.446, 145.515, 47.696, 359.134, 145.327, 529.357, 252.338, 77.716, 1163.668, 210.739, 60.384, 239.439, 347.984, 140.362, 1811.446, 72.415, 374.098], total = 7226.4439999999995
318.92496638503
